# 0. Magic

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

In [0]:
import torch
from torch import tensor
from torch import nn
import torch.nn.functional as F
from torch.utils import data 
import matplotlib.pyplot as plt

from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
from fastai.metrics import accuracy
import pickle, gzip, math, torch
import operator

# 2. Data

In [0]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [0]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

# 3. เตรียม Data

In [0]:
class Dataset(data.Dataset):
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [0]:
# x = data, m = mean, s = standard deviation
def normalize(x, m, s): 
    return (x-m)/s

In [10]:
n, m = x_train.shape
c = (y_train.max()+1).numpy()
n, m, c

(50000, 784, array(10))

In [0]:
train_mean, train_std = x_train.mean(), x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [0]:
# batch size
bs = 64
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
train_dl, valid_dl = data.DataLoader(train_ds, bs), data.DataLoader(valid_ds, bs)

# 4. สร้าง Model

Hyperparameter ของโมเดล

In [0]:
# learning rate
lr = 0.03
epoch = 10
nh = 50

ประกาศฟังก์ชันเอาไว้สร้างโมเดล

In [0]:
def get_model():
    # loss function
    loss_func = F.cross_entropy
    model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh,c))
    return model, loss_func

# 5. Training Loop

เราจะเทรนโมเดล ด้วยอัลกอริทึม [Stochastic Gradient Descent (SGD)](https://www.bualabs.com/wp-admin/post.php?post=631&action=edit) และ เก็บ Loss, Accuracy เอาไว้พล็อตกราฟ

ประกาศฟังก์ชัน fit เอาไว้เรียกเทรนเวลาที่ต้องการ

In [0]:
def fit():
    losses, metrics = [], []
    # e = epoch number
    for e in range(epoch):
        for xb, yb in train_dl:

            # Feedforward
            yhatb = model(xb)
            loss = loss_func(yhatb, yb)

            # Metrics
            acc = accuracy(yhatb, yb)
            losses.append(loss); metrics.append(acc)

            # Backpropagation
            loss.backward()

            optim.step()
            optim.zero_grad()

    plot_metrics(losses, metrics)

ประการฟัง์ชัน ไว้พล็อตกราฟ Loss และ Accuracy

In [0]:
def plot_metrics(losses, metrics):
    x = torch.arange(len(losses)).numpy()
    fig,ax = plt.subplots(figsize=(9, 9))
    ax.grid(True)
    ax.plot(x, losses, label="Loss")
    ax.plot(x, metrics, label="Accuracy")
    ax.legend(loc='upper right')

# 6. Refactor Training Loop ส่วน Metrics

การเทสว่าโมเดลทำงานเป็นอย่างไร ที่ถูกต้องเราไม่ควรเทสกับข้อมูล ใน Training Set ที่เราใช้เทรน เพราะจะทำให้ไม่ได้ค่าที่แท้จริง ที่ถูกคือเราควรเทสกับข้อมูลที่โมเดลไม่เคยเห็นมาก่อน ใน Validation Set 

[Train/Validation/Test Split คืออะไร คลิก](https://www.bualabs.com/archives/532/what-is-train-test-split-why-split-train-set-validation-set-test-set/)

In [0]:
def fit2():
    # e = epoch number
    for e in range(epoch):

        # Set Model in Train Mode
        model.train()

        for xb, yb in train_dl:
            yhatb = model(xb)
            loss = loss_func(yhatb, yb)
            loss.backward()
            optim.step()
            optim.zero_grad()

        # Set Model in Evaluation Mode
        model.eval()

        # Metrics
        with torch.no_grad():
            # tot_loss = total loss, tot_acc = total accuracy
            tot_loss, tot_acc = 0., 0.
            for xb, yb in valid_dl:
                yhatb = model(xb)
                tot_acc += accuracy(yhatb, yb)
                tot_loss += loss_func(yhatb, yb)
            nv = len(valid_ds)/bs
            print(f'epoch={e}, valid_loss={tot_loss/nv}, valid_acc={tot_acc/nv}')            
    return tot_loss/nv, tot_acc/nv
    

ลองเทรนด้วย SGD

In [38]:
model, loss_func = get_model()
optim = torch.optim.SGD(model.parameters(), lr=lr)
fit2()

epoch=0, valid_loss=0.25888174772262573, valid_acc=0.9309999942779541
epoch=1, valid_loss=0.19232112169265747, valid_acc=0.9505000114440918
epoch=2, valid_loss=0.16025273501873016, valid_acc=0.9607999920845032
epoch=3, valid_loss=0.14066414535045624, valid_acc=0.9660999774932861
epoch=4, valid_loss=0.12726172804832458, valid_acc=0.9689000248908997
epoch=5, valid_loss=0.1181999146938324, valid_acc=0.9718999862670898
epoch=6, valid_loss=0.11170954257249832, valid_acc=0.973800003528595
epoch=7, valid_loss=0.10678752511739731, valid_acc=0.975600004196167
epoch=8, valid_loss=0.10319341719150543, valid_acc=0.9760000109672546
epoch=9, valid_loss=0.10033988952636719, valid_acc=0.9761000275611877


(tensor(0.1003), tensor(0.9761))

# 7. สรุป

1. เราได้ Refactor ส่วนของ Metrics ใน Training Loop ให้ใช้ Validation Set หา Loss และ Metrics หลังจากเทรนเรียบร้อยแล้ว ในทุก ๆ Epoch
1. เรารวม Total Loss ไว้ใน tot_loss แล้วจึงหารด้วย nv (จำนวนข้อมูล / batch size) เนื่องจาก batch สุดท้ายอาจจะไม่เต็ม batch ก็ได้ เราจึงไม่สามารถหารด้วย len(valid_dl) ได้
1. เมื่อเราใช้ข้อมูลที่โมเดลไม่เคยเห็น ใน Validation Set มาเทสโมเดล แล้วได้ผลดี ทำให้เราสามารถคาดหวังได้ว่า โมเดลจะทำงานกับข้อมูลจากโลกภายนอกที่ไม่เคยเห็นมาก่อนได้ดี เช่นกัน

# Credit

* https://course.fast.ai/videos/?lesson=9
* https://www.bualabs.com/